In [1]:
import pandas as pd

In [2]:
data=pd.read_excel('ResponseGoogleForm.xlsx')

In [ ]:
data.shape

In [ ]:
data.head(8)

In [ ]:
data.tail(3)

In [ ]:
data.sample(5)

In [ ]:
#datanya secara deskriptif
data.describe()

In [ ]:
#melihat kolom yang ada
data.columns

In [ ]:
# menghitung variabel kategorik pada kolom Is_Response
data['KATEGORI'].value_counts()

In [3]:
#Create Train and Tes
from sklearn.model_selection import train_test_split

In [4]:
X=data.KELUHAN
y=data.KATEGORI

In [5]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.1, random_state=225)

In [ ]:
print('Banyak data x_train :',len(x_train))
print('Banyak data x_test  :',len(x_test))
print('Banyak data y_train :',len(y_train))
print('Banyak data y_test  :',len(y_test))

In [6]:
#Text Pre-Processing
import string

#Fungsi untuk merubah kalimat menjadi lower case dan
#menghilangkan spesial karakter/tanda baca
def lower_case(teks):
    return teks.translate(str.maketrans('','',string.punctuation)).lower().strip()
#lower_case('Saya MEmakan nasi tadi pagi !. @ #')

In [7]:
#Remove StopWord / useless word
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
def remove_stop_words(teks):
    factory = StopWordRemoverFactory()
    stopword = factory.create_stop_word_remover()
    stop = stopword.remove(teks)
    tokens = word_tokenize(stop)
    return ' '.join(map(str, tokens))
#     tokens = word_tokenize(teks)
#     listStopword =  set(stopwords.words('indonesian'))
#     listStopword.remove('tidak')
    
#     teks_hasil = []
#     for t in tokens:
#         if t not in listStopword:
#             teks_hasil.append(t)
#     teks_bersih = " ".join([word for word in teks_hasil])
#     return teks_bersih

#remove_stop_words('Siapa sih yang tidak tau Afgan')

In [8]:
#Stemming Bahasa Indonesia
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming_indo(teks):
    hasil = stemmer.stem(teks)
    return hasil

#stemming_indo('Membanggakan, menunda, mengejek')

In [9]:
#Fungsi data_preprocessing
def data_preprocessing(teks):
    hasil_lower = lower_case(teks)
    hasil_stemming= stemming_indo(hasil_lower)
    kalimat_bersih = remove_stop_words(hasil_stemming)
    
    return kalimat_bersih

In [10]:
#Tambah 1 Column Baru untu menyimpan data yg sudah dibersihkan

x=[]
for teks in data.KELUHAN:
    x.append(data_preprocessing(teks))

In [11]:
clean_text=pd.DataFrame({'clean_text':x})

In [12]:
data=pd.concat([data,clean_text],axis=1)

In [ ]:
data.head(5)

In [15]:
#Feature Extraction / Vectorizer
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfVectorizer

#tvec=TfidfVectorizer(ngram_range=(1,1))
cvec = CountVectorizer()

In [16]:
#Membuat Model. Finally Huft

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

clasifiNB = MultinomialNB()

modelKategori= Pipeline([('vectorizer',cvec)
                 ,('classifier',clasifiNB)])

modelKategori.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [ ]:
#Prediction
coba = ['Ruang lab pemrograman kotor']
hasil=modelKategori.predict(x_)
print(hasil)

In [ ]:
#Cek Akurasi
from sklearn.metrics import accuracy_score,confusion_matrix
confusion_matrix(hasil,y_test)
print('Akurasi : ',accuracy_score(hasil,y_test))

In [ ]:
import joblib

In [ ]:
#Save Model
joblib.dump(modelKategori, 'modelKategori.pkl')